# Library

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import time as tm

# Objects

In [2]:
timer = 0
ID = 0
arrive = 1

class Customer:
    
    def __init__(self,ID):
        self.__ID = ID
        self.__time = {
            'Enter':0,
            'Start_ser':0,
            'End_ser':0,
            'Exit': 0
        }
        self.__start_ser = 0
        self.__end_ser = 0
        
        self.__move_Entry_Serv = np.inf
        self.__move_Serv_Exit = np.inf
        self.__serviced = None
        
    @property
    def move_Entry_Serv(self):
        pass
    
    @property
    def move_Serv_Exit(self):
        pass

    @property
    def serviced(self):
        pass
    
    @property
    def end_of_serv(self):
        pass
    
    @property
    def start_of_serv(self):
        pass
    
    
    @move_Entry_Serv.getter
    def move_Entry_Serv(self):
        return self.__move_Entry_Serv
    
    @move_Entry_Serv.setter
    def move_Entry_Serv(self,input):
        self.__move_Entry_Serv = input
        

    
    @move_Serv_Exit.getter
    def move_Serv_Exit(self):
        return self.__move_Serv_Exit
    
    @move_Serv_Exit.setter
    def move_Serv_Exit(self,input):
        self.__move_Serv_Exit = input
        
        
        
    @serviced.getter
    def serviced(self):
        return self.__serviced
    
    @serviced.setter
    def serviced(self,input):
        self.__serviced = input
        

    @start_of_serv.getter
    def start_of_serv(self):
        return self.__start_ser
    
    @start_of_serv.setter
    def end_of_serv(self,input):
        self.__start_ser = input
        
        
    
    @end_of_serv.getter
    def end_of_serv(self):
        return self.__end_ser
    
    @end_of_serv.setter
    def end_of_serv(self,input):
        self.__end_ser = input
        
    
        
    #Kerja
        
    def act(self):
        print("im doing something here mate")
        
    def arrival(self, start):
        global arrive
        
        self.__time['Enter'] = start
        arrive += np.random.exponential(1/_lambda,1)[0]
        
    def served(self, duration):
        
        start = timer
        end = timer + duration
        
        self.__start_ser = start
        self.__end_ser = end
        
        self.__time['Start_ser'] = tm(int(start/3600), int((start/60)%60), int(start)%60 )
        self.__time['End_ser'] = tm(int(end/3600), int((end/60)%60), int(end)%60 )
        

In [3]:
class Service:
    
    def __init__(self,S_ID,C):
        self.__cap = C
        self.__S_ID = S_ID
        
        self.__start = timer
        self.__end = timer
        
        self.__occup = 0
        
    @property
    def occup(self):
        pass
        
    @property
    def cap(self):
        pass
    
    @property
    def start(self):
        pass
    
    @property
    def end(self):
        pass
    
    #Cap
    
    @cap.getter
    def cap(self):
        return self.__cap
    
    @cap.setter
    def cap(self,input):
        self.__cap = input
        
    #Start    
    
    @start.getter
    def start(self):
        return self.__start
        
    @start.setter
    def start(self,input):
        self.__start = input
        
    #End
    
    @end.getter
    def end(self):
        return self.__end
    
    @end.setter
    def end(self,input):
        self.__end = input
        
    #occup
    
    @occup.getter
    def occup(self):
        return self.__occup
    
    @occup.setter
    def occup(self,input):
        self.__occup = input
    
        
    #Kerja
        
    def serve(self,customer):
        duration = np.random.exponential(1/_mu_serv, 1)[0]
        
        customer.served(duration)
        self.__start = timer
        self.__end = timer + duration
        

# Simulator

## Engine

In [4]:
timer = 0
arrive = 1

_lambda_moving = 1/20

working_serv = 0
_mu_serv = 1/60


_lambda = 1/90
ID = 0


max_time = 1*60*60
max_time_hour = int(max_time/3600)
max_time_min = int(max_time/60)%60
max_time_second = int(max_time)%60

def sim():
    global timer
    global arrive
    global _lambda_moving
    global working_serv
    global _mu_serv
    global _lambda
    global ID
    global timer_sys
    global cust
    global q_avg, q, sys_cust, sys_cust_avg, come
    
    Work_Stat = Service(0,1)

    timer_sys = 0
    
    cust = []
    sys_cust = []
    sys_cust_avg = []

    q = []
    q_avg = []

    come = []

    while timer < max_time:


        #Set Time
        time_hour = int(timer/3600)
        time_min = int(timer/60)%60
        time_second = int(timer)%60
        time_now = tm(time_hour,time_min,time_second)

        #Arrival
        if timer > arrive:
            #Summon Entity
            cust.append(Customer(ID))
            sys_cust.append(cust[ID])

            cust[ID].arrival(time_now)

            #Move
            duration_move = np.random.exponential(1/_lambda_moving,1)[0]
            cust[ID].move_Entry_Serv = duration_move + timer
            come.append(cust[ID])

            ID += 1

        #Service
        for i in range(len(cust)):

            if cust[i] not in sys_cust:
                continue

            #Accepting Customer
            if timer >= cust[i].move_Entry_Serv and Work_Stat.end <= timer and Work_Stat.cap > Work_Stat.occup :
                Work_Stat.serve(cust[i])

                Work_Stat.occup = 1

                cust[i].move_Entry_Serv = np.inf
                cust[i].serviced = True

                if cust[i] in q: q.remove(cust[i])

            #Moving

            #Left Service
            if cust[i].end_of_serv <= timer and cust[i].serviced == True:
                cust[i].serviced = False
                Work_Stat.occup = 0
                cust[i].end_of_serv = np.inf
            #Move
                duration_move = np.random.exponential(1/_lambda_moving,1)[0]
                cust[i].move_Serv_Exit = duration_move + timer

            #Exit System
            if cust[i].move_Serv_Exit <= timer and cust[i].move_Serv_Exit != np.inf :
                cust[i].move_Serv_Exit = np.inf
                sys_cust.remove(cust[i])
                
            #Queue
            elif timer >= cust[i].move_Entry_Serv and Work_Stat.start < timer and cust[i] not in q:
                q.append(cust[i])
                
        q_avg.append(len(q))
                    #Check if Service is on going
        
        if Work_Stat.occup == 1:
            working_serv += 1

        if sys_cust:
            timer_sys += 1

        sys_cust_avg.append(len(sys_cust))

        timer += 1
        
        
sim()

## Result

In [5]:
print("==== Practical Result =====")
print(f"Service_1 utilization {round(working_serv/timer,6)*100}%")
print(f"Number of Customer {len(cust)}")
print(f"Average Customer in Queue {round(np.mean(q_avg),2)}")
print(f"Average Customer in System {round(np.mean(sys_cust_avg),2)}")

==== Practical Result =====
Service_1 utilization 38.3889%
Number of Customer 33
Average Customer in Queue 0.17
Average Customer in System 0.92


In [6]:
_lambda_tot = 1/(1/_lambda )
_mu = 1/(1/_mu_serv) #+ 1/_lambda_moving)

rho = _lambda_tot/_mu_serv

print("==== Theoritical Result =====")
print(f"Service_1 utilization {round(rho,4)*100}%")

print(f"Number of Customer {round(rho/(1 - rho),3)}")
                            
print(f"Average Customer in Queue  {(rho**2)/(1 - rho)}")

==== Theoritical Result =====
Service_1 utilization 66.67%
Number of Customer 2.0
Average Customer in Queue  1.333333333333334


# Montecarlo Simulation

In [7]:
utilization = []
sim_cust = []
sim_queue = []
sim_sys = []

for i in range(1*10**3):

    timer = 0
    arrive = 1

    _lambda_moving = 1/20
    
    working_serv = 0
    _mu_serv = 1/60
    
    
    _lambda = 1/90
    ID = 0
    
    sim()
    utilization.append(working_serv/timer)
    sim_cust.append(len(cust))
    sim_sys.append(np.mean(sys_cust_avg))
    sim_queue.append(np.mean(q_avg))

print(f"Average Utilization {np.mean(utilization)}")
print(f"Average customer in store {np.mean(sim_cust)}")
print(f"Average customer in queue {np.mean(sim_queue)}")
print(f"Average customer in system {np.mean(sim_sys)}")

Average Utilization 0.6446166666666667
Average customer in store 40.849
Average customer in queue 1.048351111111111
Average customer in system 2.144676388888889


In [8]:
universal_data = {
    'ID':[],
    'Enter':[],
    'Start_ser':[],
    'End_ser':[],
    'Exit':[]
}